# Knowledge Bases for Amazon Bedrock
## Access Control Filtering

This notebook guides users through creating and implementing access controls for Knowledge Bases on Amazon Bedrock. Amazon Bedrock is a fully managed service that provides easy access to foundation models (FMs) from leading AI companies through a single API, allowing you to build and scale generative AI applications quickly and easily.

This notebook demonstrates a practical use case for a large enterprise, AcmeCorp, showcasing how to restrict data access based on user roles using a Retrieval Augmented Generation (RAG) architecture.

### Use Case Overview
At AcmeCorp, we aim to create a Knowledge Base containing content. However, not all users have access to all data. This notebook will walk you through setting up a RAG system that restricts retrieval to only the documents a user has permission to access. We will implement this using metadata filtering with Amazon Bedrock Knowledge Bases.

### Prerequisites
***ADD info here about what has been set up via workshop studio***

### Notebook Sections
1. Amazon Cognito User Management
2. User-Corpus Association in Amazon DynamoDB
3. Dataset Download and Preparation
4. Metadata Association
5. Create OpenSearch Serverless Collection
6. Create and Configure Knowledge Base for Amazon Bedrock
7. Update AWS Lambda
8. Create and Run a Streamlit Application
9. Clean-up Resources

Each section will guide you through the process of setting up and configuring the necessary components for our metadata-filtered chat interface.

This notebook was built and tested using the `conda_python3` notebook kernel

Let's import necessary Python modules and libraries, and initialize AWS service clients required for the notebook.

In [1]:
!pip install -qU opensearch-py streamlit streamlit-cognito-auth retrying boto3 botocore

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.34.4 requires botocore==1.35.4, but you have botocore 1.35.17 which is incompatible.


In [2]:
import os
import json
import time
import uuid
import boto3
import requests
import random
from utilsmod import create_base_infrastructure, create_kb_infrastructure, updateDataAccessPolicy, createAOSSIndex, replace_vars
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from botocore.exceptions import ClientError

s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
lambda_client = boto3.client('lambda')
dynamodb_resource = boto3.resource('dynamodb')
cloudformation = boto3.client('cloudformation')
opensearch = boto3.client('opensearchserverless')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock = boto3.client("bedrock",region_name=region)
account_id = sts_client.get_caller_identity()["Account"]
cognito_client = boto3.client('cognito-idp', region_name=region)
identity_arn = session.client('sts').get_caller_identity()['Arn']
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bucket_name = 'namer-' + account_id + '-bucket'

Get outputs from Cloudformation to use later in the notebook

In [4]:
stack = cloudformation.describe_stacks(
    StackName='Namer'
)

outputs = stack["Stacks"][0]["Outputs"]
for output in outputs:
    keyName = output["OutputKey"]
    if keyName == "clientid":
        clientid = output["OutputValue"]
    elif keyName == "clientsecret":
        clientsecret = output["OutputValue"]
    elif keyName == "cognitoarn":
        cognitoarn = output["OutputValue"]
    elif keyName == "dynamotable":
        dynamotable = output["OutputValue"]
    elif keyName == "lambdafunctionarn":
        lambdafunctionarn = output["OutputValue"]
    elif keyName == "s3bucket":
        s3bucket = output["OutputValue"]
    elif keyName == "userpoolarn":
        userpoolarn = output["OutputValue"]
    elif keyName == "userpoolid":
        userpoolid = output["OutputValue"]
    elif keyName == "VPC":
        VPC = output["OutputValue"]

print("clientid: " + clientid)
print("clientsecret: " + clientsecret)
print("cognitoarn: " + cognitoarn)
print("dynamotable: " + dynamotable)
print("lambdafunctionarn: " + lambdafunctionarn)
print("s3bucket: " + s3bucket)
print("userpoolarn: " + userpoolarn)
print("userpoolid: " + userpoolid)
print("VPC: " + VPC)

clientid: 5ehs26es747p4t1qnpc0869pck
clientsecret: 1h0s7c78acooagncu7up4hbb6fik90cau0a65n5jetj7oem9moqi
cognitoarn: arn:aws:cognito-idp:us-east-1:850754977538:userpool/us-east-1_Ho4YZNl28
dynamotable: namer-850754977538-User_corpus_list_association
lambdafunctionarn: arn:aws:lambda:us-east-1:850754977538:function:namer-850754977538-lambda-function
s3bucket: namer-850754977538-bucket
userpoolarn: arn:aws:cognito-idp:us-east-1:850754977538:userpool/us-east-1_Ho4YZNl28
userpoolid: us-east-1_Ho4YZNl28
VPC: vpc-0bb775ba15b7b7690


### 0. VPC to Bedrock Endpoint

In this section, we use Amazon Virtual Private Cloud (Amazon VPC) to set up the VPC endpoint for Amazon Bedrock to facilitate private connectivity from your VPC to Amazon Bedrock.

1. On the Amazon VPC console, under Virtual private cloud in the navigation pane, choose Endpoints.
2. Choose Create endpoint.


3. For Name tag, enter bedrock-vpce.
4. Under Services, search for bedrock-runtime and select com.amazonaws.<region>.bedrock-agent-runtime.  Where region is us-west-2
5. For VPC, specify the VPC Bedrock-GenAI-Project-vpc that you created through the CloudFormation stack in the previous section.
6. In the Subnets section, and select the Availability Zones and choose the corresponding subnet IDs from the drop-down menu.
7. For Security groups, select the security group with the group name Bedrock-GenAI-Stack-VPCEndpointSecurityGroup- and description Allow TLS for VPC Endpoint.  A security group acts as a virtual firewall for your instance to control inbound and outbound traffic. Note that this VPC endpoint security group only allows traffic originating from the security group attached to your VPC private subnets, adding a layer of protection.
8. Choose Create endpoint.
9. In the Policy section, select Custom and enter the following least privilege policy to ensure only certain actions are allowed on the specified foundation model resource for a given principal (such as Lambda function IAM role).
```
{
	"Version": "2012-10-17",
	"Statement": [
		{
		    "Action": [
		        "bedrock:RetrieveAndGenerate"
		        ],
		    "Resource": [
		        "*"
		        ],
		    "Effect": "Allow",
		    "Principal": {
                "AWS": "arn:aws:iam::<accountid>:role/<region>-<accountId>-SageMaker-Execution-Namer-2024-Role"
            }
		}
	]
}
```
It may take up to 2 minutes until the interface endpoint is created and the status changes to Available. You can refresh the page to check the latest status.


### 1. Amazon Cognito User Pool: Users and Corpus Management

In this section, we'll create test users and define their associated corpus access. This simulates a real-world scenario where different users have varying levels of data access.

#### User Creation Process
We'll create users in the Amazon Cognito user pool and store their unique identifiers for later use in access control. While this notebook uses a simplified user creation process for demonstration, in a production environment, you should follow your organization's best practices for user management and security.

<div class="alert alert-block alert-warning">
<b>Warning:</b> 
<br><b>Password minimum length:</b>8 character(s)
<br><b>Password requirements</b>
<br>Contains at least 1 number
<br>Contains at least 1 special character
<br>Contains at least 1 uppercase letter
<br>Contains at least 1 lowercase letter
</div>


In [5]:
users = [
    {
        'name': 'Highway Harry',
        'email': 'highway.harry@acmecorp.com',
        'password': 'Highway.Harry.123$',
        'corpus': ['highway']
    },
    {
        'name': 'Wildlife Walter',
        'email': 'wildlife.walter@acmecorp.com',
        'password': 'Wildlife.Walter.123$',
        'corpus': ['wildlife']
    },
    {
        'name': 'Admin Amy',
        'email': 'admin.amy@acmecorp.com',
        'password': 'Admin.Amy.123$',
        'corpus': ['highway', 'wildlife']
    },
]

corpus = [
    {
        'name': 'highway',
        'description': 'document regarding highway and roadsign regulations',
        's3path': f's3://{0}/highway/'.format(bucket_name)
    },
    {
        'name': 'wildlife',
        'description': 'documents regarding fishing and hunting regulations',
        's3path': f's3://{0}/wildlife/'.format(bucket_name)
    },

]

#### Corpus-User Association
For this example, we'll create users who have access to specific information and another user that has access to all content. This setup demonstrates how you can implement role-based access control in your knowledge base system.

We use Universally Unique Identifiers (UUIDs) for corpus IDs to ensure uniqueness across our system. UUIDs are particularly useful in distributed systems where multiple parties might be generating identifiers simultaneously without coordination.

Our user setup is as follows:

1. Highway Harry: Access to highway-related documents only
2. Wildlife Walter: Access to wildlife-related documents only
3. Admin Amy: Access to both highway and wildlife documents

This structure allows us to demonstrate how different access levels can be implemented and enforced within the same knowledge base.

Each corpus (highway and wildlife) is associated with a unique UUID. When we create users, we'll associate them with these corpus IDs based on their access level. This association will later be used to filter Knowledge Base queries, ensuring users only access authorized information.

In [11]:
user_ids = []
corpus_ids = []

def create_user(user_data, user_type):
    user_ids = []
    for user in user_data:
        response = cognito_client.admin_create_user(
            UserPoolId=userpoolid,
            Username=user['email'],
            UserAttributes=[
                {'Name': 'name', 'Value': user['name']},
                {'Name': 'email', 'Value': user['email']},
                {'Name': 'email_verified', 'Value': 'true'}
            ],
            ForceAliasCreation=False,
            MessageAction='SUPPRESS'
        )
        cognito_client.admin_set_user_password(
            UserPoolId=userpoolid,
            Username=user['email'],
            Password=user['password'],
            Permanent=True
        )
        print(f"{user_type.capitalize()} created:", response['User']['Username'])
        print(f"{user_type.capitalize()} id:", response['User']['Attributes'][3]['Value'])
        user_ids.append(response['User']['Attributes'][3]['Value'])
    return user_ids

user_ids = create_user(users, 'user')
corpus_ids = [str(uuid.uuid4()) for c in corpus]

print("User IDs:", user_ids)
print("Corpus IDs:", corpus_ids)

%store user_ids corpus_ids

User created: highway.harry@acmecorp.com
User id: b41804f8-6021-7036-64d6-ab5cb55380ae
User created: wildlife.walter@acmecorp.com
User id: 94d8f4b8-7011-70b7-4158-1a1bac172c1f
User created: admin.amy@acmecorp.com
User id: f488e418-80c1-70ae-e377-2e8f817ee3f9
User IDs: ['b41804f8-6021-7036-64d6-ab5cb55380ae', '94d8f4b8-7011-70b7-4158-1a1bac172c1f', 'f488e418-80c1-70ae-e377-2e8f817ee3f9']
Corpus IDs: ['d445ea2e-ff16-4e8c-a1a3-ae3a3011cd19', '59cc5bfe-9f22-48b1-b995-0a309027b6fc']
Stored 'user_ids' (list)
Stored 'corpus_ids' (list)


Next, we'll implement this user-corpus association and create our test users in the Cognito User Pool.

### 2. User-Corpus Association in DynamoDB

This section demonstrates how to populate the pre-created DynamoDB table with user-corpus associations. This table acts as a lookup for determining which corpus (or set of documents) a user has permission to access.

#### Process Overview
1. We'll use the DynamoDB resource to interact with our table.
2. For each user, we'll create an entry in the table that lists the corpus IDs they have access to.
3. This association will later be used to filter Knowledge Base queries, ensuring users only access authorized information.

#### Implementation Details
- The table schema uses the user's ID as the partition key.
- The corpus IDs are stored as a list, allowing for multiple corpus associations per user.
- This flexible structure allows for easy updates to user permissions by modifying their corpus list.


#### Corpus-User Association
For this example, we'll create users who have access to specific information and another user that has access to all content. This setup demonstrates how you can implement role-based access control in your knowledge base system.

We use Universally Unique Identifiers (UUIDs) for corpus IDs to ensure uniqueness across our system. UUIDs are particularly useful in distributed systems where multiple parties might be generating identifiers simultaneously without coordination.

In [12]:
table = dynamodb_resource.Table(dynamotable)
corpus_mapping = [entry['name'] for entry in corpus]
with table.batch_writer() as batch:
    for corpus_list,user in enumerate(users):
        temp = []
        for corpus_id,corpuses in enumerate(corpus_mapping):
            if corpuses in user['corpus']:
                temp.append(corpus_ids[corpus_id])

        batch.put_item(
            Item={
                'user_id': user_ids[corpus_list],
                'corpus_id_list': temp
            }
        )

print('Data inserted successfully!')

Data inserted successfully!


### 3. Dataset Preparation and Upload

In this section, we'll prepare the document corpus from the 'source_transcripts' folder that will form the basis of our Knowledge Base.

#### Process Overview
1. The documents are organized into different folders, representing different data categories or access levels.
2. We'll upload these documents to our own S3 bucket for use in the Knowledge Base.

#### Implementation Details
- The documents are stored in two different folders, simulating different data categories.
- We use the boto3 S3 client to interact with the S3 buckets.
- The `upload_file` method is used to transfer files from the source to our destination bucket.

#### Considerations
In a production environment, ensure that:
  1. You have the necessary permissions to access and download the source documents.
  2. The transfer process is secure, preferably using encryption in transit.
  3. You implement error handling and logging for the file transfer process.
  4. Consider implementing a mechanism to handle large datasets, such as multipart uploads or batch processing.


In [13]:
abs_path = os.path.abspath("source_transcripts")

for root, dirs, files in os.walk(abs_path):
    for file_name in files:
        # Construct the full local path to the file
        local_file_path = os.path.join(root, file_name)
        
        # Construct the S3 key (object key) using the relative path of the file
        s3_key = os.path.relpath(local_file_path, abs_path)
        
        # Upload the file to S3
        s3_client.upload_file(local_file_path, s3bucket, s3_key)
        
        print(f'{local_file_path} uploaded successfully to {s3bucket} with key {s3_key}.')

/home/ec2-user/SageMaker/namer-summit-2024-genAI-privacy/source_transcripts/highway/23 CFR Part 655 (up to date as of 8-21-2024).pdf uploaded successfully to namer-850754977538-bucket with key highway/23 CFR Part 655 (up to date as of 8-21-2024).pdf.
/home/ec2-user/SageMaker/namer-summit-2024-genAI-privacy/source_transcripts/highway/FHWA_Strategic_Plan_05.25.23.pdf uploaded successfully to namer-850754977538-bucket with key highway/FHWA_Strategic_Plan_05.25.23.pdf.
/home/ec2-user/SageMaker/namer-summit-2024-genAI-privacy/source_transcripts/highway/Federal Highway Administration - Wikipedia.html uploaded successfully to namer-850754977538-bucket with key highway/Federal Highway Administration - Wikipedia.html.
/home/ec2-user/SageMaker/namer-summit-2024-genAI-privacy/source_transcripts/wildlife/Loon - Wikipedia.pdf uploaded successfully to namer-850754977538-bucket with key wildlife/Loon - Wikipedia.pdf.
/home/ec2-user/SageMaker/namer-summit-2024-genAI-privacy/source_transcripts/wildlife

#### Next Steps
After uploading the documents, we'll associate metadata with each file to enable fine-grained access control.

### 4. Metadata Association

This crucial step involves attaching metadata to each document in our S3 bucket. This metadata will be used to implement access control at the document level.

#### Process Overview
1. We iterate through each corpus and its corresponding files in the S3 bucket.
2. For each file, we create a metadata JSON file that includes a `corpus_id`.
3. This metadata file is uploaded alongside the original document.

#### Implementation Details
- We use the `list_objects_v2` method to get all files in each corpus folder.
- For each file, we create a metadata JSON structure with a `corpus_id`.
- The metadata file is named `<original_filename>.metadata.json`.
- We use the S3 `put_object` method to upload the metadata files.

#### Importance of Metadata
The `corpus_id` in the metadata allows us to:
1. Associate documents with specific user access levels.
2. Implement fine-grained filtering in our Knowledge Base queries.
3. Ensure users only retrieve information they're authorized to access.

#### Security Considerations
- Ensure that metadata files are treated with the same level of security as the documents they describe.
- Implement versioning on your S3 bucket to track changes to both documents and metadata.
- Consider encrypting sensitive metadata information.

In [14]:
# Loop through the corpus and their corresponding IDs
for corpuses, corpus_entry in enumerate(corpus):
    corpus_id = corpus_ids[corpuses]
    s3path = corpus_entry['s3path']
    
    # Get bucket and prefix
    # Remove 's3://' and split bucket and prefix
    path_parts = s3path.replace('s3://', '').split('/', 1)
    bucket = path_parts[0]
    prefix = path_parts[1] if len(path_parts) > 1 else ''
    
    # List all files in the S3 folder
    response = s3_client.list_objects_v2(Bucket=s3bucket, Prefix=prefix)
    if 'Contents' in response:
        files = [obj['Key'] for obj in response['Contents'] if obj['Key'] != prefix]
    else:
        files = []
    
    for file in files:
        metadata = {
            "metadataAttributes": {
                "corpus_id": corpus_id
            }
        }

        # Upload metadata file to S3
        s3_client.put_object(
            Bucket=s3bucket,
            Key=f"{file}.metadata.json",
            Body=json.dumps(metadata, indent=4),
            ContentType='application/json'
        )

#### Next Steps
With our documents and metadata in place, we'll proceed to set up the OpenSearch Serverless collection that will power our Knowledge Base.

### 5. Create OpenSearch Serverless Collection

In this section, we'll create an OpenSearch Serverless collection to be used by our Amazon Bedrock Knowledge Base. OpenSearch Serverless provides a fully managed search and analytics service that's easy to set up and scale.

We're using OpenSearch Serverless for this knowledge base implementation because it offers several advantages:
1. Automatic scaling based on workload
2. Pay-only-for-what-you-use pricing model
3. Simplified operations with no clusters to manage
4. Built-in security features

#### Process Overview
1. Create an encryption policy for the collection.
2. Create a network policy to control access to the collection.
3. Create a data access policy to manage permissions.
4. Create the OpenSearch Serverless collection itself.

#### Step 1: Create Encryption Policy
We start by creating an encryption policy that specifies how data in our collection should be encrypted. This uses AWS-owned keys for simplicity, but in a production environment, you might consider using customer-managed keys for additional control.

In [15]:
policy = '{{"Rules":[{{"ResourceType": "collection", "Resource":["collection/namer-{0}-kbcollection"]}}], "AWSOwnedKey": true}}'.format(account_id)
print(policy)

results = opensearch.create_security_policy(
    description='Public encryption access namer workshop collection',
    name='namer-' + account_id + '-kbenc',
    policy=policy,
    type='encryption'
)

print(results)

{"Rules":[{"ResourceType": "collection", "Resource":["collection/namer-850754977538-kbcollection"]}], "AWSOwnedKey": true}
{'securityPolicyDetail': {'createdDate': 1726103272974, 'description': 'Public encryption access namer workshop collection', 'lastModifiedDate': 1726103272974, 'name': 'namer-850754977538-kbenc', 'policy': {'Rules': [{'Resource': ['collection/namer-850754977538-kbcollection'], 'ResourceType': 'collection'}], 'AWSOwnedKey': True}, 'policyVersion': 'MTcyNjEwMzI3Mjk3NF8x', 'type': 'encryption'}, 'ResponseMetadata': {'RequestId': 'ae15309d-d6ea-4cf4-a734-751b78e2716e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ae15309d-d6ea-4cf4-a734-751b78e2716e', 'date': 'Thu, 12 Sep 2024 01:07:53 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '375', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


#### Step 2: Create Network Policy
Next, we create a network policy that defines how the collection can be accessed. For this demo, we're allowing public access, but in a production environment, you'd want to restrict this to specific VPCs or IP ranges.

In [16]:
policy = '''[{{"Rules": [{{"ResourceType": "dashboard", 
    "Resource": ["collection/namer-{0}-kbcollection"]}}, 
    {{"ResourceType": "collection", "Resource": ["collection/namer-{0}-kbcollection"]}}], 
    "AllowFromPublic": true}}]'''.format(account_id)
print(policy)
results = opensearch.create_security_policy(
    description='Public network access namer workshop collection',
    name='namer-{0}-kbnet'.format(account_id),
    policy=policy,
    type='network'
)

print(results)

[{"Rules": [{"ResourceType": "dashboard", 
    "Resource": ["collection/namer-850754977538-kbcollection"]}, 
    {"ResourceType": "collection", "Resource": ["collection/namer-850754977538-kbcollection"]}], 
    "AllowFromPublic": true}]
{'securityPolicyDetail': {'createdDate': 1726103275521, 'description': 'Public network access namer workshop collection', 'lastModifiedDate': 1726103275521, 'name': 'namer-850754977538-kbnet', 'policy': [{'Rules': [{'Resource': ['collection/namer-850754977538-kbcollection'], 'ResourceType': 'dashboard'}, {'Resource': ['collection/namer-850754977538-kbcollection'], 'ResourceType': 'collection'}], 'AllowFromPublic': True}], 'policyVersion': 'MTcyNjEwMzI3NTUyMV8x', 'type': 'network'}, 'ResponseMetadata': {'RequestId': '9f04f378-3c62-44ac-919d-22f6a6dbe769', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9f04f378-3c62-44ac-919d-22f6a6dbe769', 'date': 'Thu, 12 Sep 2024 01:07:55 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length'

#### Step 3: Create Data Access Policy
The data access policy determines who can perform what actions on the collection. We're granting broad permissions to our IAM role for this demo, but in practice, you'd want to follow the principle of least privilege.

In [17]:
policy = '''[{{"Rules": [{{"Resource": ["collection/namer-{0}-kbcollection"], 
                           "Permission": ["aoss:CreateCollectionItems", "aoss:UpdateCollectionItems", "aoss:DescribeCollectionItems"], 
                           "ResourceType": "collection"}}, 
                          {{"ResourceType": "index", "Resource": ["index/namer-{0}-kbcollection/*"], 
                           "Permission": ["aoss:CreateIndex", "aoss:DescribeIndex", "aoss:ReadDocument", "aoss:WriteDocument", "aoss:UpdateIndex", "aoss:DeleteIndex"]}}], 
                "Principal": ["arn:aws:iam::{0}:role/Namer-{0}-KBRole"]}}]'''.format(account_id)
print(policy)
results = opensearch.create_access_policy(
    description='Data access policy for the NAMER summit',
    name='namer-{0}-kbaccess'.format(account_id),
    policy=policy,
    type='data'
)

[{"Rules": [{"Resource": ["collection/namer-850754977538-kbcollection"], 
                           "Permission": ["aoss:CreateCollectionItems", "aoss:UpdateCollectionItems", "aoss:DescribeCollectionItems"], 
                           "ResourceType": "collection"}, 
                          {"ResourceType": "index", "Resource": ["index/namer-850754977538-kbcollection/*"], 
                           "Permission": ["aoss:CreateIndex", "aoss:DescribeIndex", "aoss:ReadDocument", "aoss:WriteDocument", "aoss:UpdateIndex", "aoss:DeleteIndex"]}], 
                "Principal": ["arn:aws:iam::850754977538:role/Namer-850754977538-KBRole"]}]


Now that we have our policies we can create the OpenSearch Serverless Collection

#### Step 4: Create OpenSearch Serverless Collection
Finally, we create the actual OpenSearch Serverless collection, applying the policies we've just created.

In [18]:
results = opensearch.create_collection(
    description='KB AOSS Collection',
    name='namer-{0}-kbcollection'.format(account_id),
    type='VECTORSEARCH'
)

print(results)

{'createCollectionDetail': {'arn': 'arn:aws:aoss:us-east-1:850754977538:collection/9uv78m67b35qupg12sf3', 'createdDate': 1726103280676, 'description': 'KB AOSS Collection', 'id': '9uv78m67b35qupg12sf3', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1726103280676, 'name': 'namer-850754977538-kbcollection', 'standbyReplicas': 'ENABLED', 'status': 'CREATING', 'type': 'VECTORSEARCH'}, 'ResponseMetadata': {'RequestId': '1b379c45-2e10-4757-92bc-8211ba6db5a3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1b379c45-2e10-4757-92bc-8211ba6db5a3', 'date': 'Thu, 12 Sep 2024 01:08:01 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '358', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


Creating the collection takes some time so we will check to see if it has been created yet

In [19]:
response = opensearch.list_collections(collectionFilters={'name':'namer-{0}-kbcollection'.format(account_id)})
collection_id = response["collectionSummaries"][0]["id"]
collection_arn = response["collectionSummaries"][0]["arn"]
print("Creating OpenSearch Collection")
while opensearch.list_collections(collectionFilters={'name':'namer-{0}-kbcollection'.format(account_id)})["collectionSummaries"][0]["status"] != "ACTIVE":
    time.sleep(30)
    print("in progress....")

print("Collection created")


Creating OpenSearch Collection
in progress....
in progress....
in progress....
in progress....
in progress....
in progress....
in progress....
Collection created


In [20]:
indexName = "namer-{0}-kb-acl-index".format(account_id)
print("Index name:",indexName)
%store indexName

Index name: namer-850754977538-kb-acl-index
Stored 'indexName' (str)


In [21]:
# Adding the current role to the collection's data access policy
data_access_policy_name = 'namer-{0}-kbaccess'.format(account_id)

response = opensearch.get_access_policy(
    name=data_access_policy_name,
    type='data'
)
policy_version = response["accessPolicyDetail"]["policyVersion"]
existing_policy = response['accessPolicyDetail']['policy']
updated_policy = existing_policy.copy()
updated_policy[0]['Principal'].append('arn:aws:iam::{0}:role/{1}-{0}-SageMaker-Execution-Namer-2024-Role'.format(account_id, boto3.session.Session().region_name))
updated_policy = str(updated_policy).replace("'", '"')

response = opensearch.update_access_policy(
    description='dataAccessPolicy',
    name=data_access_policy_name,
    policy=updated_policy,
    policyVersion=policy_version,
    type='data'
)
print(response)

time.sleep(60) # Changes to the data access policy might take a bit to update
print("Finished adding the role")

{'accessPolicyDetail': {'createdDate': 1726103277992, 'description': 'dataAccessPolicy', 'lastModifiedDate': 1726103522828, 'name': 'namer-850754977538-kbaccess', 'policy': [{'Rules': [{'Resource': ['collection/namer-850754977538-kbcollection'], 'Permission': ['aoss:CreateCollectionItems', 'aoss:UpdateCollectionItems', 'aoss:DescribeCollectionItems'], 'ResourceType': 'collection'}, {'Resource': ['index/namer-850754977538-kbcollection/*'], 'Permission': ['aoss:CreateIndex', 'aoss:DescribeIndex', 'aoss:ReadDocument', 'aoss:WriteDocument', 'aoss:UpdateIndex', 'aoss:DeleteIndex'], 'ResourceType': 'index'}], 'Principal': ['arn:aws:iam::850754977538:role/Namer-850754977538-KBRole', 'arn:aws:iam::850754977538:role/us-east-1-850754977538-SageMaker-Execution-Namer-2024-Role']}], 'policyVersion': 'MTcyNjEwMzUyMjgyOF8y', 'type': 'data'}, 'ResponseMetadata': {'RequestId': 'b244f21d-3424-426f-a879-212b52b371cf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b244f21d-3424-426f-a879-212

In [22]:
# Set up AWS authentication
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, region, service)

# Define index settings and mappings
index_settings = {
    "settings": {
        "index.knn": "true"
    },
    "mappings": {
        "properties": {
            "vector": {
                "type": "knn_vector",
                "dimension": 1024,
                 "method": {
                     "name": "hnsw",
                     "engine": "faiss",
                     "space_type": "innerproduct",
                     "parameters": {
                         "ef_construction": 512,
                         "m": 16
                     },
                 },
             },
            "text": {
                "type": "text"
            },
            "text-metadata": {
                "type": "text"
            }
        }
    }
}

# Build the OpenSearch client
host = f"{collection_id}.{region}.aoss.amazonaws.com"
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# Create index
response = oss_client.indices.create(index=indexName, body=json.dumps(index_settings))
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'namer-850754977538-kb-acl-index'}


#### Next Steps
With our OpenSearch Serverless collection in place, we're ready to create and configure our Knowledge Base in Amazon Bedrock.

### 6. Create and Configure Knowledge Base for Amazon Bedrock

This section covers the process of creating and configuring a Knowledge Base using Amazon Bedrock. This Knowledge Base will utilize the OpenSearch Serverless collection we just created to store and retrieve document embeddings.

#### Process Overview
1. Select an embedding model for the Knowledge Base.
2. Create the Knowledge Base using Amazon Bedrock.
3. Create a data source linking to our S3 bucket.
4. Start an ingestion job to populate the Knowledge Base.

#### Step 1: Select Embedding Model
We'll use Amazon Titan Embeddings V2 for this Knowledge Base. This model will convert our text documents into vector embeddings for efficient similarity search.

Important: Ensure you have enabled access to Amazon Titan Embeddings V2 in the Amazon Bedrock Console before proceeding.

In [23]:
embeddingModelArn = "arn:aws:bedrock:{}::foundation-model/amazon.titan-embed-text-v2:0".format(region)

#### Step 2: Create Knowledge Base
We use the `create_knowledge_base` API call to set up our Knowledge Base. Key configuration points include:
- Specifying the embedding model.
- Linking to our OpenSearch Serverless collection.
- Setting up field mappings for text, metadata, and vector data.

In [25]:
results = bedrock_agent_client.create_knowledge_base(
    description='Test KB Deployment',
    knowledgeBaseConfiguration={
        'type': 'VECTOR',
        'vectorKnowledgeBaseConfiguration': {
            'embeddingModelArn': embeddingModelArn
        }
    },
    name='namer-{0}-knowledge-base'.format(account_id),
    roleArn='arn:aws:iam::{0}:role/Namer-{0}-KBRole'.format(account_id),
    storageConfiguration={
        'opensearchServerlessConfiguration': {
            'collectionArn': collection_arn,
            'fieldMapping': {
                'metadataField': 'text-metadata',
                'textField': 'text',
                'vectorField': 'vector'
            },
            'vectorIndexName': indexName
        },
        'type': 'OPENSEARCH_SERVERLESS'
    }
)

print(results)

kb_id = results["knowledgeBase"]["knowledgeBaseId"]

{'ResponseMetadata': {'RequestId': '44dc5c3f-99b4-4767-9055-d59227cfd7d1', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Thu, 12 Sep 2024 01:16:07 GMT', 'content-type': 'application/json', 'content-length': '895', 'connection': 'keep-alive', 'x-amzn-requestid': '44dc5c3f-99b4-4767-9055-d59227cfd7d1', 'x-amz-apigw-id': 'd98RrHOwIAMEunA=', 'x-amzn-trace-id': 'Root=1-66e240d7-4266fc823215def6771dfc2a'}, 'RetryAttempts': 0}, 'knowledgeBase': {'createdAt': datetime.datetime(2024, 9, 12, 1, 16, 7, 288697, tzinfo=tzlocal()), 'description': 'Test KB Deployment', 'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:850754977538:knowledge-base/LL85IR1HOW', 'knowledgeBaseConfiguration': {'type': 'VECTOR', 'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}}, 'knowledgeBaseId': 'LL85IR1HOW', 'name': 'namer-850754977538-knowledge-base', 'roleArn': 'arn:aws:iam::850754977538:role/Namer-850754977538-KBRole', 'status': 'CREA

#### Step 3: Create Data Source
We create a data source that points to our S3 bucket containing the documents and metadata. This step uses the `create_data_source` API call.

In [26]:
results = bedrock_agent_client.create_data_source(
    dataSourceConfiguration={
        's3Configuration': {
            'bucketArn': 'arn:aws:s3:::{0}'.format(s3bucket),
        },
        'type': 'S3',
    },
    description='KB Data Source',
    knowledgeBaseId=kb_id,
    name='namer-{0}-kb_datasource'.format(account_id),

    vectorIngestionConfiguration={
        'chunkingConfiguration': {
            'chunkingStrategy': 'FIXED_SIZE',
            'fixedSizeChunkingConfiguration': {
                'maxTokens': 300,
                'overlapPercentage': 20
            },
        }
    }
)

print(results)

datasource_id = results["dataSource"]["dataSourceId"]

{'ResponseMetadata': {'RequestId': 'd13a3cbb-5fef-428f-b3e8-12f5a9316786', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 12 Sep 2024 01:16:11 GMT', 'content-type': 'application/json', 'content-length': '567', 'connection': 'keep-alive', 'x-amzn-requestid': 'd13a3cbb-5fef-428f-b3e8-12f5a9316786', 'x-amz-apigw-id': 'd98SXEXJoAMEhrA=', 'x-amzn-trace-id': 'Root=1-66e240db-0eaeeb38145aabdf3ec74be2'}, 'RetryAttempts': 0}, 'dataSource': {'createdAt': datetime.datetime(2024, 9, 12, 1, 16, 11, 698021, tzinfo=tzlocal()), 'dataDeletionPolicy': 'DELETE', 'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::namer-850754977538-bucket'}, 'type': 'S3'}, 'dataSourceId': '62KMPYRLNZ', 'description': 'KB Data Source', 'knowledgeBaseId': 'LL85IR1HOW', 'name': 'namer-850754977538-kb_datasource', 'status': 'AVAILABLE', 'updatedAt': datetime.datetime(2024, 9, 12, 1, 16, 11, 698021, tzinfo=tzlocal()), 'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': '

#### Step 4: Start Ingestion Job
Finally, we initiate an ingestion job to process our documents, generate embeddings, and store them in the OpenSearch collection. We monitor the job status to ensure successful completion.

#### Important Considerations
- The ingestion process can take some time, depending on the volume of documents.
- Ensure your IAM roles have the necessary permissions for all these operations.
- In a production environment, consider setting up monitoring and alerting for ingestion jobs.

In [27]:
ingestion_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=kb_id,
    dataSourceId=datasource_id,
    description='Initial Ingestion'
)

In [28]:
status = bedrock_agent_client.get_ingestion_job(
    knowledgeBaseId=ingestion_job_response["ingestionJob"]["knowledgeBaseId"],
    dataSourceId=ingestion_job_response["ingestionJob"]["dataSourceId"],
    ingestionJobId=ingestion_job_response["ingestionJob"]["ingestionJobId"]
)["ingestionJob"]["status"]
print(status)
while status not in ["COMPLETE", "FAILED", "STOPPED"]:
    status = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId=ingestion_job_response["ingestionJob"]["knowledgeBaseId"],
        dataSourceId=ingestion_job_response["ingestionJob"]["dataSourceId"],
        ingestionJobId=ingestion_job_response["ingestionJob"]["ingestionJobId"]
    )["ingestionJob"]["status"]
    print(status)
    time.sleep(30)
print("Waiting for changes to take place in the vector database")
time.sleep(30) # Wait for all changes to take place
print("COMPLETE")

IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
COMPLETE
Waiting for changes to take place in the vector database
COMPLETE


#### Testing the Knowledge Base
After ingestion is complete, we demonstrate how to use the `retrieve` and `retrieve_and_generate` APIs to query our Knowledge Base. These examples show how to:
- Filter queries based on corpus IDs (implementing our access control).
- Retrieve relevant document chunks.
- Generate responses using a language model based on the retrieved information.

The `retrieve` API is used when you only need to fetch relevant passages from the Knowledge Base. It's useful when you want to see the raw, unprocessed information or when you plan to process the retrieved information yourself.

The `retrieve_and_generate` API goes a step further. It not only retrieves relevant passages but also uses a specified language model to generate a coherent response based on those passages. This is particularly useful when you want to provide a more human-friendly, synthesized answer to a query.

<div class="alert alert-block alert-warning">
<b>Warning:</b> Make sure you have enabled Anthropic Claude 3 Sonnet access in the Amazon Bedrock Console (model access). 
</div>

In [29]:
# retrieve and generate API
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Which office do I submit for golden eagle permits?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0".format(region),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5,
                    "filter": {
                        "equals": {
                            "key": "corpus_id",
                            "value": corpus_ids[1]
                        }
                    }
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

To obtain permits for golden eagle activities such as scientific collecting, exhibition, Native American religious purposes, depredation, nest take, and incidental take, you should submit your application to the "Migratory Bird Permit Program Office" in the region where you reside.



In this second example we are going to use the **retrieve API**. This API queries the knowledge base and retrieves relavant information from it, it does not generate the response.

In [30]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":3,
            "filter": {
                 "equals": {
                    "key": "corpus_id",
                    "value": corpus_ids[1]
                        }
                    }
                } 
            },
    retrievalQuery={
        'text': "Which office do I submit for golden eagle permits?"   
        }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents
# each list has content,location,score,metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

Chunk 1:  Endangered Species Act permit applications for the import or export of native endangered and threatened species may be obtained from the Division of Management Authority in accordance with paragraph (b)(3) of this section.   (5) You may obtain applications for bald and golden eagle permits (50 CFR part 22) and migratory bird permits (50 CFR part 21), except for banding and marking permits, from, and you may submit completed applications to, the “Migratory Bird Permit Program Office” in the Region in which you reside. For addresses of the regional offices, see 50 CFR 2.2, or go to: http://www.fws.gov/ migratorybirds/mbpermits/Addresses.html.   (c) Time notice. The Service will process all applications as quickly as possible. However, we cannot guarantee final action within the time limit you request. You should ensure that applications for permits for marine mammals and/or endangered and threatened species are postmarked at least 90 calendar days prior to the requested effecti

#### Next Steps
With our Knowledge Base set up and populated, we'll move on to updating our Lambda function to use the latest SDK, ensuring compatibility with our metadata filtering approach.

### 7. Update AWS Lambda Function

At the time of developing this notebook, the default Boto3 version available in Lambda with Python 3.12 doesn't include metadata filtering capabilities for Bedrock Knowledge Bases. To overcome this limitation, we'll create and attach an AWS Lambda Layer with the latest Boto3 version.

Updating the Lambda layer is necessary to ensure that our function has access to the latest AWS SDK features, particularly those related to Bedrock Knowledge Bases and metadata filtering. 

#### Process Overview
1. Create a directory for the Lambda layer.
2. Install the latest Boto3 and Botocore versions in this directory.
3. Create a ZIP file of the installed packages.
4. Publish a new Lambda layer using this ZIP file.
5. Attach the new layer to our existing Lambda function.

In [25]:
# can we have the lambda layer already attached to the lambda function?  What is we have it prebuilt?

#### Prerequisites
This section requires the `zip` package to be installed at the system level. You can check if it's installed by running the `!zip` command. If it's not available, you'll need to install it using the appropriate package manager for your system (e.g., `apt-get` for Debian-based systems or `yum` for RHEL-based systems).

In [31]:
!mkdir latest-sdk-layer
%cd latest-sdk-layer
!pip install -qU boto3 botocore -t python/lib/python3.12/site-packages/
!zip -rq latest-sdk-layer.zip .
%cd ..

mkdir: cannot create directory ‘latest-sdk-layer’: File exists
/home/ec2-user/SageMaker/namer-summit-2024-genAI-privacy/latest-sdk-layer
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.34.4 requires botocore==1.35.4, but you have botocore 1.35.17 which is incompatible.
jupyter-server 2.14.2 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
sphinx 8.0.2 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.
sphinx 8.0.2 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
/home/ec2-user/SageMaker/namer-summit-2024-genAI-privacy


In [32]:
def publish_lambda_layer(layer_name, description, zip_file_path, compatible_runtimes):
    with open(zip_file_path, 'rb') as f:
        response = lambda_client.publish_layer_version(
            LayerName=layer_name,
            Description=description,
            Content={
                'ZipFile': f.read(),
            },
            CompatibleRuntimes=compatible_runtimes
        )
    return response['LayerVersionArn']

In [33]:
layer_name = 'latest-sdk-layer'
description = 'Layer with the latest boto3 version.'
zip_file_path = 'latest-sdk-layer/latest-sdk-layer.zip'
compatible_runtimes = ['python3.12']

In [34]:
layer_version_arn = publish_lambda_layer(layer_name, description, zip_file_path, compatible_runtimes)
print("Layer version ARN:", layer_version_arn)

Layer version ARN: arn:aws:lambda:us-east-1:850754977538:layer:latest-sdk-layer:2


In [35]:
try:
    # Add the layer to the Lambda function
    lambda_client.update_function_configuration(
        FunctionName=lambdafunctionarn,
        Layers=[layer_version_arn]
    )
    print("Layer added to the Lambda function successfully.")

except ClientError as e:
    print(f"Error adding layer to Lambda function: {e.response['Error']['Message']}")
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Layer added to the Lambda function successfully.


add function to the VPC and update the policy.

add 
```    
    {
            "Effect": "Deny",
            "Action": [
                "bedrock:RetrieveAndGenerate"
            ],
            "Resource": "*",
            "Condition": {
                "ForAnyValue:StringEquals": {
                    "aws:sourceVpce": [
                        "vpce-0f88bae3249d891d9" <vpce id of the bedrock-agent-runtime>
                    ]
                }
            }
        }
```
    to lambda exectution role

#### Next Steps
With our Lambda function now using the latest Boto3 version, we're ready to create a user interface to interact with our metadata-filtered Knowledge Base.

### 8. Create and Run Streamlit Application

To showcase the interaction between users and our Knowledge Base with metadata filtering, we'll create a simple web application using Streamlit. This popular Python library allows us to quickly build interactive data apps.

#### Application Overview
The Streamlit app will:
1. Authenticate users using Amazon Cognito.
2. Retrieve the user's associated corpus IDs from DynamoDB.
3. Allow users to enter search queries.
4. Use our Lambda function to query the Knowledge Base with appropriate metadata filters.
5. Display the results to the user.

#### Implementation Steps
1. We define the Streamlit app in a Python file (`app.py`).
2. The app uses the `streamlit_cognito_auth` package for user authentication.
3. After authentication, it retrieves the user's Cognito sub (unique identifier) and associated corpus IDs.
4. The search functionality invokes our Lambda function, passing the user's query and corpus IDs for filtering.

In [36]:
%%writefile app.py
import os
import boto3
import json
import requests
import streamlit as st
from streamlit_cognito_auth import CognitoAuthenticator

pool_id = "<<replace_pool_id>>"
app_client_id = "<<replace_app_client_id>>"
app_client_secret = "<<replace_app_client_secret>>"
kb_id = "<<replace_kb_id>>"
lambda_function_arn = '<<replace_lambda_function_arn>>'
dynamo_table = '<<replace_dynamo_table_name>>'

authenticator = CognitoAuthenticator(
    pool_id=pool_id,
    app_client_id=app_client_id,
    app_client_secret= app_client_secret,
    use_cookies=False
)

is_logged_in = authenticator.login()

if not is_logged_in:
    st.stop()

def logout():
    authenticator.logout()

def get_user_sub(userpoolid, username):
    cognito_client = boto3.client('cognito-idp')
    try:
        response = cognito_client.admin_get_user(
            UserPoolId=pool_id,
            Username=authenticator.get_username()
        )
        sub = None
        for attr in response['UserAttributes']:
            if attr['Name'] == 'sub':
                sub = attr['Value']
                break
        return sub
    except cognito_client.exceptions.UserNotFoundException:
        print("User not found.")
        return None

def get_corpus_ids(user_id):
    dynamodb = boto3.client('dynamodb')
    response = dynamodb.query(
        TableName=dynamo_table,
        KeyConditionExpression='user_id = :user_id',
        ExpressionAttributeValues={
            ':user_id': {'S': user_id}
        }
    )
    print(response)
    corpus_id_list = []  # Initialize the list
    for item in response['Items']:
        corpus_ids = item.get('corpus_id_list', {}).get('L', [])
        corpus_id_list.extend([corpus_id['S'] for corpus_id in corpus_ids])
    return corpus_id_list

def search_transcript(user_id, kb_id, text, corpus_ids):
    # Initialize the Lambda client
    lambda_client = boto3.client('lambda')

    # Payload for the Lambda function
    payload = json.dumps({
        "userId": sub,
        "knowledgeBaseId": kb_id,
        "text": text, 
        "corpusIds": corpus_ids
    }).encode('utf-8')

    try:
        # Invoke the Lambda function
        response = lambda_client.invoke(
            FunctionName=lambda_function_arn,
            InvocationType='RequestResponse',
            Payload=payload
        )

        # Process the response
        if response['StatusCode'] == 200:
            response_payload = json.loads(response['Payload'].read().decode('utf-8'))
            return response_payload
        else:
            # Handle error response
            return {'error': 'Failed to fetch data'}

    except Exception as e:
        # Handle exception
        return {'error': str(e)}

sub = get_user_sub(pool_id, authenticator.get_username())
print(sub)
corpus_ids = get_corpus_ids(sub)
print(corpus_ids)

# Application Front

with st.sidebar:
    st.header("User Information")
    st.markdown("## User")
    st.text(authenticator.get_username())
    st.markdown("## User Id")
    st.text(sub)
    # selected_patient = st.selectbox("Select a patient (or 'All' for all patients)", ['All'] + patient_ids)
    st.button("Logout", "logout_btn", on_click=logout)

st.header("Corpus Search Tool")

# Text input for the search query
query = st.text_input("Enter your search query:")

if st.button("Search"):
    if query:
        # Perform search
        corpus_ids_filter = corpus_ids
        results = search_transcript(sub, kb_id, query, corpus_ids_filter)
        print(results)
        if results:
            st.subheader("Search Results:")
            st.markdown(results["body"], unsafe_allow_html=True)
        else:
            st.write("No matching results found in corpus.")
    else:
        st.write("Please enter a search query.")

Overwriting app.py


In [37]:
replace_vars("app.py", userpoolid, clientid, clientsecret, kb_id, lambdafunctionarn, dynamotable)

#### Running the Application
To run the Streamlit app:
1. Execute the provided command in your notebook environment.
2. Access the app using the URL provided, which will vary depending on your environment (SageMaker Studio or SageMaker Notebook).

In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.16.18.10:8501
  External URL: http://34.236.55.223:8501

b41804f8-6021-7036-64d6-ab5cb55380ae
{'Items': [{'corpus_id_list': {'L': [{'S': 'd445ea2e-ff16-4e8c-a1a3-ae3a3011cd19'}]}, 'user_id': {'S': 'b41804f8-6021-7036-64d6-ab5cb55380ae'}}], 'Count': 1, 'ScannedCount': 1, 'ResponseMetadata': {'RequestId': 'P9THHKSO27K1EORCM3QDCRHI4NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Thu, 12 Sep 2024 01:23:17 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '165', 'connection': 'keep-alive', 'x-amzn-requestid': 'P9THHKSO27K1EORCM3QDCRHI4NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1284113190'}, 'RetryAttempts': 0}}
['d445ea2e-ff16-4e8c-a1a3-ae3a3011cd19']
b41804f8-6021-7036-64d6-ab5cb55380ae
{'Items': [{'corpus_id_list': {'L': [{'S': 'd445ea2e-ff16-4e8c-a1a3-ae3a3011cd19'}]}, 'user_id': {'S': 'b41804f8-60

#### Important Notes
- Use the email and password of the users you defined earlier in the notebook to log in to the application.
- Once logged in, you can query the Knowledge Base. The results will be filtered based on your user's permissions.

If you are executing this notebook on SageMaker Studio you can access the Streamlit application in the following url. 

```
https://<<STUDIOID>>.studio.<<REGION>>.sagemaker.aws/jupyterlab/default/proxy/8501/
```

If you are executing this notebook on a SageMaker Notebook you can access the Streamlit application in the following url. 

```
https://<<NOTEBOOKID>>.notebook.<<REGION>>.sagemaker.aws/proxy/8501/
```

In [ ]:
https://us-west-2-431615879134-sagemaker-execution-namer-2024-notebook.notebook.us-west-2.sagemaker.aws/proxy/8501/

#### Next Steps
After testing the application, proceed to the clean-up section to remove all created resources and avoid unnecessary costs.

### 9. Clean-up Resources

This final section guides you through the process of deleting all the resources created during this workshop. This step is crucial to avoid incurring unnecessary AWS costs.

#### Important: Before proceeding, ensure you have stopped the Streamlit application running in the previous step.

#### Clean-up Process
1. Delete all objects in the S3 bucket.
2. Delete the CloudFormation stacks that were created.

#### Implementation Details
- We use the S3 client to list and delete all objects in our bucket.
- We iterate through our CloudFormation stacks, checking their status and deleting them if they exist.
- The script waits for each stack deletion to complete before moving to the next one.

#### Execution Time
The clean-up process typically takes about 2-3 minutes to complete.

#### Important Considerations
- Double-check that all resources have been properly deleted by reviewing the AWS Console.
- In a production environment, consider implementing a more robust clean-up process, possibly as part of your infrastructure-as-code setup.
- Ensure that you have the necessary permissions to delete all created resources.

#### Final Notes
- Always be cautious when deleting resources. Ensure you're in the correct AWS account and region.
- If you plan to recreate this setup later, consider saving key configuration details or automating the entire process.

In [ ]:
# Delete all objects in the S3 bucket
try:
    response = s3_client.list_objects_v2(Bucket=s3bucket)
    if 'Contents' in response:
        for obj in response['Contents']:
            s3_client.delete_object(Bucket=s3bucket, Key=obj['Key'])
        print(f"All objects in {s3bucket} have been deleted.")
except Exception as e:
    print(f"Error deleting objects from {s3bucket}: {e}")

# Delete the Knowledge Base
try:
    bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
    print(f"Knowledge Base {kb_id} deleted successfully.")
except Exception as e:
    print(f"Error deleting Knowledge Base: {e}")

# Delete the OpenSearch Serverless collection
try:
    opensearch.delete_collection(id=collection_id)
    print(f"OpenSearch Serverless collection {collection_id} deleted successfully.")
except Exception as e:
    print(f"Error deleting OpenSearch Serverless collection: {e}")

# Delete the OpenSearch Serverless security policies
policy_names = [f'namer-{account_id}-kbenc', f'namer-{account_id}-kbnet']
policy_types = ['encryption', 'network']

for name, types in zip(policy_names, policy_types):
    try:
        opensearch.delete_security_policy(name=name, type=types)
        print(f"OpenSearch Serverless {types} policy {name} deleted successfully.")
    except Exception as e:
        print(f"Error deleting OpenSearch Serverless {types} policy {name}: {e}")
        
try:
    opensearch.delete_access_policy(name=data_access_policy_name, type='data')
    print(f"OpenSearch Serverless data policy {data_access_policy_name} deleted successfully.")
except Exception as e:
    print(f"Error deleting OpenSearch Serverless data policy {data_access_policy_name}: {e}")

# Delete the Lambda layer
try:
    lambda_client.delete_layer_version(LayerName=layer_name, VersionNumber=int(layer_version_arn.split(':')[-1]))
    print(f"Lambda layer {layer_name} deleted successfully.")
except Exception as e:
    print(f"Error deleting Lambda layer: {e}")

# Delete users from Cognito User Pool
for user in users:
    try:
        cognito_client.admin_delete_user(
            UserPoolId=userpoolid,
            Username=user['email']
        )
        print(f"User {user['email']} deleted from Cognito User Pool.")
    except Exception as e:
        print(f"Error deleting user {user['email']} from Cognito User Pool: {e}")

# Delete items from DynamoDB table
table = dynamodb_resource.Table(dynamotable)
try:
    with table.batch_writer() as batch:
        for user_id in user_ids:
            batch.delete_item(Key={'user_id': user_id})
    print("All items deleted from DynamoDB table.")
except Exception as e:
    print(f"Error deleting items from DynamoDB table: {e}")

print("Clean-up process completed. Please check the AWS Console to ensure all resources have been properly deleted.")